In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,precision_score,f1_score
from sklearn.model_selection import train_test_split
from keras.applications.xception import Xception
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, concatenate,GRU, Embedding, GlobalMaxPooling2D
from keras.models import Model
from keras.preprocessing.image import img_to_array, load_img
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Load text data

In [4]:
train_data = pd.read_csv('task_informative_text_img_train.csv')
test_data = pd.read_csv('task_informative_text_img_test.csv')
val_data = pd.read_csv('task_informative_text_img_dev.csv')

In [5]:
X1_text=[]
Y1_text=[]
X2_text=[]
Y2_text=[]
X3_text=[]
Y3_text=[]
# train
for i in range(0,13608):
    if train_data['label_text'][i]==train_data['label_image'][i]:
        X1_text.append(train_data['tweet_text'][i])
        Y1_text.append(train_data['label_text'][i])

In [6]:
for i in range(0,2237):
    if test_data['label_text'][i]==test_data['label_image'][i]:
        X2_text.append(test_data['tweet_text'][i])
        Y2_text.append(test_data['label_text'][i])

In [7]:
for i in range(0,2237):
    if val_data['label_text'][i]==val_data['label_image'][i]:
        X3_text.append(val_data['tweet_text'][i])
        Y3_text.append(val_data['label_text'][i])
# Tokenize text data
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X1_text)
sequences_train = tokenizer.texts_to_sequences(X1_text)
sequences_test = tokenizer.texts_to_sequences(X2_text)
sequences_val = tokenizer.texts_to_sequences(X3_text)
x_train = pad_sequences(sequences_train, maxlen=max_len)
x_test = pad_sequences(sequences_test, maxlen=max_len)
x_val = pad_sequences(sequences_val, maxlen=max_len)

In [8]:
X1_image=[]
Y1_image=[]
X2_image=[]
Y2_image=[]
X3_image=[]
Y3_image=[]
# train
for i in range(0,13608):
    if train_data['label_text'][i]==train_data['label_image'][i]:
        X1_image.append(train_data['image'][i])
        Y1_image.append(train_data['label_image'][i])

In [9]:
for i in range(0,2237):
    if test_data['label_text'][i]==test_data['label_image'][i]:
        X2_image.append(test_data['image'][i])
        Y2_image.append(test_data['label_image'][i])

In [10]:
for i in range(0,2237):
    if val_data['label_text'][i]==val_data['label_image'][i]:
        X3_image.append(val_data['image'][i])
        Y3_image.append(val_data['label_image'][i])
# Load image data
img_width, img_height = 100, 100
num_classes = 2
img_train = []
img_test = []
img_val = []

In [11]:
for filename in X1_image:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_train.append(img_array)

/opt/anaconda/envs/wmlce/lib/python3.6/site-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [12]:
for filename in X2_image:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_test.append(img_array)

In [13]:
for filename in X3_image:
    img = load_img( filename, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_val.append(img_array)

In [14]:
img_train = np.array(img_train)
img_test = np.array(img_test)
img_val = np.array(img_val)

Define gru model

In [15]:
gru_input = Input(shape=(max_len,))
embedding_layer = Embedding(max_words, 128)(gru_input)
gru_layer = GRU(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
gru_output = Dense(num_classes, activation='sigmoid')(gru_layer)
gru_model = Model(inputs=gru_input, outputs=gru_output)

Define Xception model

In [16]:
xception_input = Input(shape=(img_width, img_height, 3))
xception_model = Xception(weights='imagenet', include_top=False, input_tensor=xception_input, pooling='max')
xception_output = Dense(num_classes, activation='sigmoid')(xception_model.output)
xception_model = Model(inputs=xception_input, outputs=xception_output)

Combine gru and Xception models with intermediate fusion

In [17]:
print(np.shape(gru_model.output))
print(np.shape(xception_model.output))
combined_input = concatenate([gru_model.output, xception_model.output])
fusion_output = Dense(num_classes, activation='sigmoid')(combined_input)
fusion_model = Model(inputs=[gru_model.input, xception_model.input], outputs=fusion_output)

(None, 2)
(None, 2)


Compile the fusion model

In [18]:
fusion_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Train the fusion model

In [19]:
tempTr=[]
tempTe=[]
tempDe=[]

In [20]:
for i in range(0, len(Y1_text)):
    if Y1_text[i] == 'informative':
        tempTr.append(0)
    else:
        tempTr.append(1)
for i in range(0, len(Y2_text)):
    if Y2_text[i] == 'informative':
        tempTe.append(0)
    else:
        tempTe.append(1)
for i in range(0, len(Y3_text)):
    if Y3_text[i] == 'informative':
        tempDe.append(0)
    else:
        tempDe.append(1)
labels_train = tempTr
labels_test = tempTe
labels_val = tempDe
y_train = to_categorical(labels_train, num_classes=num_classes)
y_test = to_categorical(labels_test, num_classes=num_classes)
y_val = to_categorical(labels_val, num_classes=num_classes)

In [22]:
history = fusion_model.fit([x_train, img_train], y_train,
                           epochs=10, batch_size=40,
                           validation_data=([x_val, img_val], y_val))

Train on 9601 samples, validate on 1573 samples
Epoch 1/10
9601/9601 [==============================] - 563s 59ms/step - loss: 0.6085 - accuracy: 0.6989 - val_loss: 0.5984 - val_accuracy: 0.6888
Epoch 2/10
9601/9601 [==============================] - 563s 59ms/step - loss: 0.5919 - accuracy: 0.7164 - val_loss: 0.5937 - val_accuracy: 0.7152
Epoch 3/10
9601/9601 [==============================] - 526s 55ms/step - loss: 0.5773 - accuracy: 0.7299 - val_loss: 0.6175 - val_accuracy: 0.6910
Epoch 4/10
9601/9601 [==============================] - 495s 52ms/step - loss: 0.5656 - accuracy: 0.7442 - val_loss: 0.5576 - val_accuracy: 0.7502
Epoch 5/10
9601/9601 [==============================] - 494s 51ms/step - loss: 0.5462 - accuracy: 0.7621 - val_loss: 0.5822 - val_accuracy: 0.7301
Epoch 6/10
9601/9601 [==============================] - 493s 51ms/step - loss: 0.5304 - accuracy: 0.7752 - val_loss: 0.5822 - val_accuracy: 0.6834
Epoch 7/10
9601/9601 [==============================] - 491s 51ms/step

Evaluate the fusion model

In [23]:
score = fusion_model.evaluate([x_test, img_test], y_test, verbose=0)

Print accuracy, precision, and F1 score

In [24]:
y_pred = fusion_model.predict([x_test, img_test])
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
acc = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Accuracy: {:.2f}%".format(acc*100))
print("Precision: {:.2f}%".format(precision*100))
print("F1 Score: {:.2f}%".format(f1*100))

Accuracy: 67.60%
Precision: 75.59%
F1 Score: 68.51%


In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.60      0.71      1030
           1       0.50      0.83      0.63       504

    accuracy                           0.68      1534
   macro avg       0.69      0.72      0.67      1534
weighted avg       0.76      0.68      0.69      1534

